In [1]:
from __future__ import division
import nltk 
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import *
from textblob.classifiers import NaiveBayesClassifier
from sklearn.cross_validation import KFold
from nltk.classify.naivebayes import NaiveBayesClassifier
from gensim import corpora, models, similarities
from nltk.corpus import wordnet as wn
import urllib
import urllib2
from alchemyapi import AlchemyAPI
alchemyapi = AlchemyAPI()

In [2]:
'''
Reading the Dataset (ISEAR Dataset)
'''
Data = pd.read_csv('ISEAR.csv',header=None)
'''
36 - Class Label
40 - Sentence
'''

'\n36 - Class Label\n40 - Sentence\n'

In [3]:
'''
Emotion Labels
'''
emotion_labels = ['joy', 'fear', 'anger', 'sadness', 'disgust', 'shame', 'guilt']
# emotion_labels = ['joy', 'fear', 'anger', 'sadness', 'disgust']

In [4]:
'''
Negation words
'''
negation_words = ['not', 'neither', 'nor', 'but', 'however', 'although', 'nonetheless', 'despite', 'except', 'even though', 'yet']

In [5]:
'''
Returns a list of all corresponding class labels
'''
def class_labels(emotions):
    labels = []
    labelset = []
    exclude = []
    for i in range(len(emotions)):
#         labels.append(e)
#         labelset.append([e])
        if emotions[i] not in ['shame','guilt']:
            labels.append(e)
            labelset.append([e])
        else:
            exclude.append(i)
    return labels, labelset, exclude

In [6]:
'''
Removes unnecessary characters from sentences
'''
def removal(sentences):
    sentence_list = []
    count = 0
#     for sen in sentences:
#         count += 1
#         print count
#         print sen
#         print type(sen)
    s = nltk.word_tokenize(sentences)
    characters = ["á", "\xc3", "\xa1", "\n", ",", ".", "[", "]", ""]
    l = []
    for t in s:
        if t not in characters:
            l.append(t)
    return l
#     new = ' '.join([i for i in s if not [e for e in characters if e in i]])
#     print new
#     sentence_list.append(new)
#     return sentence_list

In [7]:
'''
POS-TAGGER, returns NAVA words
'''
def pos_tag(sentences):
    tags = [] #have the pos tag included
    nava_sen = []
    pt = nltk.pos_tag(sentences)
#     for s in sentences:
#     s_token = nltk.word_tokenize(sentences)
#     pt = nltk.pos_tag(s_token)
    nava = []
    nava_words = []
    for t in pt:
        if t[1].startswith('NN') or t[1].startswith('JJ') or t[1].startswith('VB') or t[1].startswith('RB'):
            nava.append(t)
            nava_words.append(t[0])
    return nava, nava_words
#     tags.append(nava)
#     nava_sen.append(nava_words)
#     return tags, nava_sen

In [8]:
'''
Performs stemming
'''
def stemming(sentences):
    sentence_list = []
    sen_string = []
    sen_token = []
    stemmer = PorterStemmer()
    i = 0
#     for sen in sentences:
#         print i,
    i += 1
    st = ""
    for word in sentences:
        word_l = word.lower()
        if len(word_l) >= 3:
            st += stemmer.stem(word_l) + " "
    sen_string.append(st)
    w_set = nltk.word_tokenize(st)
    sen_token.append(w_set)
    w_text = nltk.Text(w_set)
    sentence_list.append(w_text)
    return w_text, st, w_set
#     return sentence_list, sen_string, sen_token

In [9]:
'''
Write to file
'''
def write_to_file(filename, text):
    o = open(filename,'w')
    o.write(str(text))
    o.close()

In [10]:
'''
Creating the dataframe
'''
def create_frame(Data):
    labels = []
#     sentences = []
#     sen_string = []
#     sen_token =[]
    sen = []
    sen_s = []
    sen_t = []
    labelset = []
    for i in range(len(Data)):
        if i >= 0:
#             print i,
            emotion = Data[0][i]
            sit = Data[1][i]
#             if emotion not in ['shame', 'guilt']:
            labels.append(emotion)
            labelset.append([emotion])
            sent = removal(sit)
            nava, sent_pt = pos_tag(sent)
            sentences, sen_string, sen_token = stemming(sent_pt)
            sen.append(sentences)
            sen_s.append(sen_string)
            sen_t.append(sen_token)
#     labels, labelset, exclude = class_labels(emotions[1:])
#     sent = removal(sit[1:], exclude)
#     nava, sent_pt = pos_tag(sent)
#     sentences, sen_string, sen_token = stemming(sent_pt)
    frame = pd.DataFrame({0 : labels,
                          1 : sen,
                          2 : sen_s,
                          3 : sen_t,
                          4 : labelset})
    return frame, sen_t, labels, sen_s

In [11]:
c, st, labels, senten = create_frame(Data)

In [12]:
c

,0,1,2,3,4
0,joy,"(day, feel, close, partner, other, friend, fee...",day feel close partner other friend feel peac ...,"[day, feel, close, partner, other, friend, fee...",[joy]
1,fear,"(time, imagin, someon, love, contact, seriou, ...",time imagin someon love contact seriou ill eve...,"[time, imagin, someon, love, contact, seriou, ...",[fear]
2,anger,"(had, been, obvious, unjustli, treat, had, pos...",had been obvious unjustli treat had possibl el...,"[had, been, obvious, unjustli, treat, had, pos...",[anger]
3,sadness,"(think, short, time, live, relat, period, life...",think short time live relat period life think ...,"[think, short, time, live, relat, period, life...",[sadness]
4,disgust,"(gather, found, involuntarili, sit, next, peop...",gather found involuntarili sit next peopl expr...,"[gather, found, involuntarili, sit, next, peop...",[disgust]
5,shame,"(realiz, wa, direct, feel, discont, partner, w...",realiz wa direct feel discont partner way wa t...,"[realiz, wa, direct, feel, discont, partner, w...",[shame]
6,guilt,"(feel, guilti, realiz, consid, materi, thing, ...",feel guilti realiz consid materi thing more im...,"[feel, guilti, realiz, consid, materi, thing, ...",[guilt]
7,joy,"(girlfriend, had, taken, exam, went, parent, p...",girlfriend had taken exam went parent place,"[girlfriend, had, taken, exam, went, parent, p...",[joy]
8,fear,"(first, time, realiz, mean, death)",first time realiz mean death,"[first, time, realiz, mean, death]",[fear]
9,anger,"(car, overtak, forc, drive, road)",car overtak forc drive road,"[car, overtak, forc, drive, road]",[anger]


In [13]:
'''
Reads the emotion representative words file
'''
def readfile(filename):
    f = open(filename,'r')
    representative_words = []
    for line in f.readlines():
        characters = ["\n", " ", "\r", "\t"]
        new = ''.join([i for i in line if not [e for e in characters if e in i]])
        representative_words.append(new)
    return representative_words

In [14]:
'''
Makes a list of all words semantically related to an emotion and Stemming
'''
def affect_wordlist(words):
    affect_words = []
    stemmer = PorterStemmer()
    for w in words:
        w_l = w.lower()
        word_stem = stemmer.stem(w_l)
        if word_stem not in affect_words:
            affect_words.append(word_stem)
    return affect_words

In [15]:
'''
Creating an emotion wordnet
'''
def emotion_word_set(emotions):
    word_set = {}
    for e in emotions:
        representative_words = readfile(e)
        wordlist = affect_wordlist(representative_words)
        word_set[e] = wordlist
    return word_set

In [16]:
'''
Lexicon based approach - Check for lexicons
'''
def lexicon_based(sentences, word_set):
    text_vector = []
    for sen in sentences:
        s_vector = []
        for word in sen:
            w_vector = {}
            for emo in word_set:
                if word in word_set[emo]:
#                     print word
                    try:
                        if emo not in w_vector[word]:
                            w_vector[word].append(emo)
                    except KeyError:
                        w_vector[word] = [emo]
            if w_vector:
                s_vector.append(w_vector)
        if not s_vector:
            text_vector.append(s_vector)
        else:
            text_vector.append(s_vector)
    return text_vector

In [17]:
'''
Lexicon based approach - Classify based on lexicons
'''
def classify_lexicon(text_vector, labels, emotion_labels):
    count = 0
    total = 0
    for j in range(len(text_vector)):
        sen = text_vector[j]
        sen_emo = np.empty(len(emotion_labels))
        sen_emo.fill(0)
        if sen:
            total += 1
            w_emo = []
            for word in sen:
                emotions =  word.values()[0][0]
#                 print emotions, type(emotions), j
                w_emo.append(emotions)
                i = emotion_labels.index(emotions)
                sen_emo[i] += 1
#             print sen_emo
            winner = np.argwhere(sen_emo == np.amax(sen_emo))
            indices = winner.flatten().tolist()
            for i in indices:
                if emotion_labels[i] == labels[j]:
                    count += 1
                    break
#                 else:
#                     print j, text_vector[j]
    accuracy = count/len(text_vector)
    tot_accuracy = count/total
    return accuracy, tot_accuracy

In [18]:
e = emotion_word_set(emotion_labels)
l = lexicon_based(c[1],e) 
a, b = classify_lexicon(l, c[0], emotion_labels)
print a, b

0.227514635444 0.540113708149


In [19]:
'''
Calculate pmi
'''
def pmi(x, y, sentences):
    count_x = 1
    count_y = 1
    count_xy = 1
    for sen in sentences:
        if x and y in sentences:
            count_xy += 1
            count_x += 1
            count_y += 1
        if x in sentences:
            count_x += 1
        if y in sentences:
            count_y += 1
        result = count_xy/(count_x * count_y)
    return result

In [26]:
print a*100, '%'
print b*100, "%"

22.7514635444 %
54.0113708149 %


In [27]:
'''
Emotion Detector
'''
# c = create_frame(Data)
# emo_word_net = emotion_word_set(emotion_set)

'\nEmotion Detector\n'

In [28]:
'''
Getting synonyms from wordnet synsets
'''
def get_synonyms():
    syn = {}
    for e in emotion_labels:
        jw = wn.synsets(e)
        for s in jw:
            v = s.name()
            try:
                syn[e].append(wn.synset(v).lemma_names())
            except KeyError:
                syn[e] = wn.synset(v).lemma_names()

In [29]:
'''
Creating training/testing set for Naive Bayes classifier TextBlob -- Not used
'''
def create_dataset_textblob(sentences, emotions):
    train = []
    sen = []
    emo = []
    for s in sentences:
        sen.append(s)
    for e in emotions:
        emo.append(e)
    for i in range(len(sen)):
        s = sen[i]
        e = emo[i]
        train.append((str(s), e))
    return train

In [30]:
'''
Testing for Naive Bayes Classifier
'''
def testing(cl, test):
    for s, e in test:
        r = cl.classify(s)
        print s, e, r
        if r == e:
            print "*"

In [79]:
'''
Create dataset for nltk Naive Bayes
'''
def create_data(sentence, emotion):
    data = []
    for i in range(len(sentence)):
        sen = []
        for s in sentence[i]:
            sen.append(str(s))
        emo = emotion[i]
        data.append((sen, emo))
    return data

In [80]:
'''
Get all words in dataset
'''
def get_words_in_dataset(dataset):
    all_words = []
    for (words, sentiment) in dataset:
        all_words.extend(words)
    return all_words

In [81]:
'''
Getting frequency dist of words
'''
def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

In [82]:
'''
Extacting features
'''
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [83]:
'''
Create test data
'''
def create_test(sentence, emotion):
    data = []
    sen = []
    emo = []
    for s in sentence:
        sen.append(str(s))
    for e in emotion:
        emo.append(e)
    for i in range(len(sen)):
        temp = []
        temp.append(sen[i])
        temp.append(emo[i])
        data.append(temp)
    return data

In [84]:
'''
Classifier
'''
def classify_dataset(data):
    return \
        classifier.classify(extract_features(nltk.word_tokenize(data)))

In [85]:
'''
Get accuracy
'''
def get_accuracy(test_data, classifier):
    total = accuracy = float(len(test_data))
    for data in test_data:
        if classify_dataset(data[0]) != data[1]:
#             print data, classify_dataset(data[0]), data[1]
            accuracy -= 1
    print('Total accuracy: %f%% (%d/20).' % (accuracy / total * 100, accuracy))
    final = accuracy / total * 100
    return final

In [86]:
# Create training and testing data
sen = c[3]
emo = c[0]
l = len(c[3])
limit = (9*l)//10
sente = c[2]
Data = create_data(sen[:limit], emo[:limit])
test_data = create_test(sente[limit:], emo[limit:]) 

In [87]:
# extract the word features out from the training data
word_features = get_word_features(\
                    get_words_in_dataset(Data))

In [88]:
# get the training set and train the Naive Bayes Classifier
training_set = nltk.classify.util.apply_features(extract_features, Data)
classifier = NaiveBayesClassifier.train(training_set)

In [89]:
Naive_accu = get_accuracy(test_data, classifier)

Total accuracy: 63.164894% (475/20).


In [42]:
length = len(st)
train_size = (9 * length) // 10
train_data = st[:train_size]
test_data = senten[train_size:]

In [43]:
gensim_dict = corpora.Dictionary(train_data)

In [44]:
gensim_dict.save('corpus.dict')

In [45]:
gen_token = gensim_dict.token2id

In [46]:
corpusmm = [gensim_dict.doc2bow(text) for text in train_data]
corpora.MmCorpus.serialize('corpus.mm', corpusmm)

In [47]:
gen_c = corpora.MmCorpus('corpus.mm')

In [48]:
tfidf = models.TfidfModel(gen_c)

In [49]:
corpus_tfidf = tfidf[gen_c]

In [50]:
dictionary = corpora.Dictionary.load('corpus.dict')
lsi = models.LsiModel(corpus_tfidf, id2word=gensim_dict, num_topics=7) # initialize an LSI transformation
corpus_lsi = lsi[corpus_tfidf]

In [51]:
index = similarities.MatrixSimilarity(lsi[gen_c])

In [52]:
lsi.save('corpus.lsi') 
index.save('corpus.index')
# lsi = models.LsiModel.load('corpus.lsi')

In [53]:
# length = len(corpus_lsi)
# train_size = (9 * length) // 10
# train_data = corpus_lsi[:train_size]
# test_data = corpus_lsi[train_size:]

In [54]:
test_labels = labels[train_size:]
train_labels = labels[:train_size]

In [55]:
def semantic_sim(test_data):
    total = 0
    for count, doc in enumerate(test_data):
        vec_bow = dictionary.doc2bow(doc.lower().split())
        vec_lsi = lsi[vec_bow]
        sims = index[vec_lsi]
        ans = sorted(enumerate(sims), key=lambda item: -item[1])[0]
        ind = ans[0] 
        if train_labels[ind] == test_labels[count]:
            total += 2
    avg = total/count
    return avg

In [56]:
sem_accu = semantic_sim(test_data)

In [ ]:
url = 'http://gateway-a.watsonplatform.net/calls/text/TextGetEmotion'
params = urllib.urlencode({
  'apikey': '15ce4bd07b66f9e000a15383777870c0afb383fb',
  'text': 'I am excited',
  'outputMode': 'json'
})
response = urllib2.urlopen(url, params).read()

In [ ]:
response

In [57]:
print "Accuracy using Lexical Component - First Phase  ", a*100, '%'
print "Accuracy using Lexical Component - Second Phase  ", b*100, "%"

Accuracy using Lexical Component - First Phase   22.7514635444 %
Accuracy using Lexical Component - Second Phase   54.0113708149 %


In [92]:
print "Accuracy using Naive Bayes Component  ", Naive_accu, "%"

Accuracy using Naive Bayes Component   63.164893617 %


In [93]:
print "Accuracy using semantic similarity component ", sem_accu*100, "%"

Accuracy using semantic similarity component  53.7949400799 %


In [1]:
len(train_size)

NameError: name 'train_size' is not defined